# MACD
This notebook use MACD strategy to backtest the performance

In [4]:
# Run it when crypto-backtest is updated, and then restart the kernel
!pip uninstall crypto-backtest -y
!pip install git+https://github.com/BullionBear/crypto-backtest.git

Found existing installation: crypto-backtest 0.0.15
Uninstalling crypto-backtest-0.0.15:
  Successfully uninstalled crypto-backtest-0.0.15
  Cloning https://github.com/BullionBear/crypto-backtest.git to /tmp/pip-req-build-h33mlhqp
  Running command git clone --filter=blob:none --quiet https://github.com/BullionBear/crypto-backtest.git /tmp/pip-req-build-h33mlhqp
  Resolved https://github.com/BullionBear/crypto-backtest.git to commit 23bbd3c677aa481bf2f0fda79ee3788f940fe492
  Preparing metadata (setup.py) ... done
  Created wheel for crypto-backtest: filename=crypto_backtest-0.0.16-py3-none-any.whl size=10171 sha256=89f78b0b67c2b1c9f7b50f29d1deea310505f0474ab85dc13a721177b4a8b129
  Stored in directory: /tmp/pip-ephem-wheel-cache-00lq_gv_/wheels/14/d1/5f/193347d44b51670dd287000ac535cdb7a8fc3f84e426532382
Successfully built crypto-backtest

[notice] A new release of pip is available: 23.2.1 -> 24.0
[notice] To update, run: pip install --upgrade pip


In [5]:
import backtest
backtest.__version__

'develop_0.0.13+'

In [7]:
from backtest.components.core import ZipKLineIterator
from backtest.components import KLineMarket

from datetime import datetime, timezone

start_time = datetime(2023, 7, 1, tzinfo=timezone.utc).timestamp() * 1000
end_time = datetime(2023, 9, 1, tzinfo=timezone.utc).timestamp() * 1000
data_source = '/home/yite/crypto_data/binance/data'
kline_iterator = ZipKLineIterator('BTCUSDT', start_time, end_time, data_source)
market = KLineMarket(kline_iterator)

In [ ]:
from backtest.components.core import SMAFiltration

fast_div = SMAFiltration(5)
slow_div = SMAFiltration(10)
for _ in range(10 + 1):
    if market.is_kline():
        kline = market.get_kline()
        fast_div.put(kline)
        slow_div.put(kline)
print("fast_div"